In [1]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

DATA_PATH = "/kaggle/input/fyp-dataset/ck/ck/"
TOP_EMOTIONS = ["Angry", "Fear", "Happy", "Sadness", "Surprise"]

TRAIN_SIZE = 0.80
NUM_CLASSES = len(TOP_EMOTIONS)

2024-03-19 09:34:44.341743: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 09:34:44.341873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 09:34:44.539402: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Define input size
input_shape = (48, 48, 1)

In [2]:
# Define input size
input_shape = (48, 48, 1)

total_images = 0
for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        count = len(os.listdir(os.path.join(DATA_PATH, dir_)))
        print(f"{dir_} has {count} number of images")
        total_images += count

print(f"\ntotal images are {total_images}")

img_arr = []
img_label = []
label_to_text = {}
label = 0

for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        for f in os.listdir(os.path.join(DATA_PATH, dir_)):
            img = cv2.imread(os.path.join(DATA_PATH, dir_, f), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, input_shape[:2])  # Resize the image
            img_arr.append(np.expand_dims(img, axis=-1))
            img_label.append(label)
        print(f"loaded {dir_} images to numpy arrays...")
        label_to_text[label] = dir_
        label += 1

img_arr = np.array(img_arr)
img_label = np.array(img_label)
img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

Surprise has 249 number of images
Fear has 75 number of images
Angry has 135 number of images
Sadness has 84 number of images
Happy has 207 number of images

total images are 750
loaded Surprise images to numpy arrays...
loaded Fear images to numpy arrays...
loaded Angry images to numpy arrays...
loaded Sadness images to numpy arrays...
loaded Happy images to numpy arrays...


In [3]:
# Normalize pixel values
img_arr = img_arr / 255.

X_train, X_test, y_train, y_test = train_test_split(img_arr, img_label,
                                                    shuffle=True, stratify=img_label,
                                                    train_size=TRAIN_SIZE, random_state=42)

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (48, 48, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.3344 - loss: 1.5307 - val_accuracy: 0.5667 - val_loss: 1.3572
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.5715 - loss: 1.2454 - val_accuracy: 0.7133 - val_loss: 0.8322
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.7764 - loss: 0.6961 - val_accuracy: 0.8467 - val_loss: 0.5026
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.8831 - loss: 0.4225 - val_accuracy: 0.8733 - val_loss: 0.3582
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9347 - loss: 0.2565 - val_accuracy: 0.9267 - val_loss: 0.2375
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.9447 - loss: 0.1802 - val_accuracy: 0.9600 - val_loss: 0.1519
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.9654 - loss: 0.1189 - val_accuracy: 0.9533 - val_loss: 0.1506
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.9719 - loss: 0.1088 - val_accuracy: 0.9467 - v

# Define input size
input_shape = (64, 64, 1)

In [5]:
# Define input size
input_shape = (64, 64, 1)

total_images = 0
for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        count = len(os.listdir(os.path.join(DATA_PATH, dir_)))
        print(f"{dir_} has {count} number of images")
        total_images += count

print(f"\ntotal images are {total_images}")

img_arr = []
img_label = []
label_to_text = {}
label = 0

for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        for f in os.listdir(os.path.join(DATA_PATH, dir_)):
            img = cv2.imread(os.path.join(DATA_PATH, dir_, f), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, input_shape[:2])  # Resize the image
            img_arr.append(np.expand_dims(img, axis=-1))
            img_label.append(label)
        print(f"loaded {dir_} images to numpy arrays...")
        label_to_text[label] = dir_
        label += 1

img_arr = np.array(img_arr)
img_label = np.array(img_label)
img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

Surprise has 249 number of images
Fear has 75 number of images
Angry has 135 number of images
Sadness has 84 number of images
Happy has 207 number of images

total images are 750
loaded Surprise images to numpy arrays...
loaded Fear images to numpy arrays...
loaded Angry images to numpy arrays...
loaded Sadness images to numpy arrays...
loaded Happy images to numpy arrays...


In [6]:
# Normalize pixel values
img_arr = img_arr / 255.

X_train, X_test, y_train, y_test = train_test_split(img_arr, img_label,
                                                    shuffle=True, stratify=img_label,
                                                    train_size=TRAIN_SIZE, random_state=42)

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (64, 64, 1)
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - accuracy: 0.3545 - loss: 1.5124 - val_accuracy: 0.6000 - val_loss: 1.1579
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.7172 - loss: 0.9478 - val_accuracy: 0.8000 - val_loss: 0.5690
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.8454 - loss: 0.4929 - val_accuracy: 0.8800 - val_loss: 0.3428
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - accuracy: 0.9230 - loss: 0.2622 - val_accuracy: 0.9400 - val_loss: 0.2085
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.9718 - loss: 0.1316 - val_accuracy: 0.9400 - val_loss: 0.1581
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 106ms/step - accuracy: 0.9732 - loss: 0.1015 - val_accuracy: 0.9800 - val_loss: 0.1088
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.9751 - loss: 0.0736 - val_accuracy: 0.9867 - val_loss: 0.0942
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - accuracy: 0.9923 - loss: 0.0364 - val_accuracy: 0.9600 - val_loss: 0

# Define input size
input_shape = (128, 128, 1)

In [8]:
# Define input size
input_shape = (128, 128, 1)

total_images = 0
for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        count = len(os.listdir(os.path.join(DATA_PATH, dir_)))
        print(f"{dir_} has {count} number of images")
        total_images += count

print(f"\ntotal images are {total_images}")

img_arr = []
img_label = []
label_to_text = {}
label = 0

for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        for f in os.listdir(os.path.join(DATA_PATH, dir_)):
            img = cv2.imread(os.path.join(DATA_PATH, dir_, f), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, input_shape[:2])  # Resize the image
            img_arr.append(np.expand_dims(img, axis=-1))
            img_label.append(label)
        print(f"loaded {dir_} images to numpy arrays...")
        label_to_text[label] = dir_
        label += 1

img_arr = np.array(img_arr)
img_label = np.array(img_label)
img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

Surprise has 249 number of images
Fear has 75 number of images
Angry has 135 number of images
Sadness has 84 number of images
Happy has 207 number of images

total images are 750
loaded Surprise images to numpy arrays...
loaded Fear images to numpy arrays...
loaded Angry images to numpy arrays...
loaded Sadness images to numpy arrays...
loaded Happy images to numpy arrays...


In [9]:
# Normalize pixel values
img_arr = img_arr / 255.

X_train, X_test, y_train, y_test = train_test_split(img_arr, img_label,
                                                    shuffle=True, stratify=img_label,
                                                    train_size=TRAIN_SIZE, random_state=42)

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [10]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (128, 128, 1)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 375ms/step - accuracy: 0.3153 - loss: 1.7358 - val_accuracy: 0.5800 - val_loss: 1.2711
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 361ms/step - accuracy: 0.5937 - loss: 1.1192 - val_accuracy: 0.8000 - val_loss: 0.7072
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 391ms/step - accuracy: 0.8325 - loss: 0.5658 - val_accuracy: 0.8667 - val_loss: 0.3852
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 367ms/step - accuracy: 0.9169 - loss: 0.2816 - val_accuracy: 0.9400 - val_loss: 0.2672
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 370ms/step - accuracy: 0.9408 - loss: 0.2342 - val_accuracy: 0.8733 - val_loss: 0.3174
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 11s 393ms/step - accuracy: 0.9526 - loss: 0.1518 - val_accuracy: 0.9533 - val_loss: 0.1484
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 368ms/step - accuracy: 0.9769 - loss: 0.0884 - val_accuracy: 0.9800 - val_loss: 0.0804
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 363ms/step - accuracy: 0.9919 - loss: 0.0362 - val_accur

# Define input size
input_shape = (256, 256, 1)

In [14]:
# Define input size
input_shape = (256, 256, 1)

total_images = 0
for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        count = len(os.listdir(os.path.join(DATA_PATH, dir_)))
        print(f"{dir_} has {count} number of images")
        total_images += count

print(f"\ntotal images are {total_images}")

img_arr = []
img_label = []
label_to_text = {}
label = 0

for dir_ in os.listdir(DATA_PATH):
    if dir_ in TOP_EMOTIONS:
        for f in os.listdir(os.path.join(DATA_PATH, dir_)):
            img = cv2.imread(os.path.join(DATA_PATH, dir_, f), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, input_shape[:2])  # Resize the image
            img_arr.append(np.expand_dims(img, axis=-1))
            img_label.append(label)
        print(f"loaded {dir_} images to numpy arrays...")
        label_to_text[label] = dir_
        label += 1

img_arr = np.array(img_arr)
img_label = np.array(img_label)
img_label = OneHotEncoder().fit_transform(img_label.reshape(-1, 1)).toarray()  # Convert to dense array

Surprise has 249 number of images
Fear has 75 number of images
Angry has 135 number of images
Sadness has 84 number of images
Happy has 207 number of images

total images are 750
loaded Surprise images to numpy arrays...
loaded Fear images to numpy arrays...
loaded Angry images to numpy arrays...
loaded Sadness images to numpy arrays...
loaded Happy images to numpy arrays...


In [15]:
# Normalize pixel values
img_arr = img_arr / 255.

X_train, X_test, y_train, y_test = train_test_split(img_arr, img_label,
                                                    shuffle=True, stratify=img_label,
                                                    train_size=TRAIN_SIZE, random_state=42)

def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [17]:
# Train and evaluate the model
print(f"Training model with input shape: {input_shape}")
model = create_cnn_model(input_shape, NUM_CLASSES)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Training model with input shape: (256, 256, 1)
Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.2796 - loss: 7.6255 - val_accuracy: 0.3333 - val_loss: 1.4661
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.4838 - loss: 1.3280 - val_accuracy: 0.7133 - val_loss: 0.8289
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.7828 - loss: 0.7087 - val_accuracy: 0.8533 - val_loss: 0.4601
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.8585 - loss: 0.4420 - val_accuracy: 0.8600 - val_loss: 0.4165
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.9340 - loss: 0.1980 - val_accuracy: 0.9267 - val_loss: 0.2686
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.9655 - loss: 0.1193 - val_accuracy: 0.9333 - val_loss: 0.2097
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.9668 - loss: 0.1064 - val_accuracy: 0.9133 - val_loss: 0.1714
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.9739 - 